In [17]:
# IMPORT STATEMENTS - DO NOT CHANGE

import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn import tree
import graphviz
from matplotlib import pyplot as plt

In [18]:
# READ IN CSV OF EPIC RUNS 

# CHANGE PATH TO THE APPROPRIATE PATH FOR WHERE YOUR DATA IS LOCATED 

df = pd.read_csv("C:\\Users\\Research\\Documents\\Tara_Fall_2019\\HyperCube\\EPIC_CZcube_PARM2.csv")


# TEST DATA 
field = pd.read_excel("C:\\Users\\Research\\Documents\\Tara_Fall_2019\\HyperCube\\yield_2000_2017.xlsx")

# SITE DATA 
site = pd.read_excel("C:\\Users\\Research\\Documents\\Tara_Fall_2019\\HyperCube\\czech_sites.xlsx")
simuid = pd.read_csv("C:\\Users\\Research\\Documents\\Tara_Fall_2019\\HyperCube\\field_w_simuid.csv")

In [19]:
# PREP TEST DATA 
col = field.iloc[:, 7:-1]

field['min_yield'] = col.min(axis=1)
field['max_yield'] = col.max(axis=1)

In [20]:
# pull out texture, precip, residues management, P application, manure application
site_vars = site[['Site', 'Prec (mm)', 'Soil texture']]

In [21]:
# texture conversions 
tex_data = {'Soil texture':  ['loam', 'clay loam', 'sandy loam', 'loamy sand'],
        'Sand': [40, 35, 65, 82],
         'Silt': [40, 30, 25, 13],
        'Clay': [20, 35, 10, 5]
        }

tex = pd.DataFrame (tex_data, columns = ["Soil texture", 'Sand', 'Silt', 'Clay'])

In [22]:
site_vars = site_vars.drop_duplicates()

In [23]:
simuid["Site_Name"].replace({"Lednice": "Lednice", "Uhersk? Ostroh": "Uherský Ostroh", 
                            'V?rovany': 'Věrovany', 'Ivanovice': 'Ivanovice ', '??slav': 'Čáslav',
                           '?atec': 'Žatec', 'Hn?v?eves': 'Hněvčeves', 'Pust? Jakartice': 'Pusté Jakartice', 'Kostelec': 'Kostelec',
                           'Praha - Ruzyn?': 'Praha - Ruzyně', 'Chrastava': 'Chrastava', 'Sta?kov': 'Staňkov', 'Jarom??ice': 'Jaroměřice', 'Trutnov': 'Trutnov',
                           'Hradec n. Svit.': 'Hradec n. Svit.', 'Hora??ovice': 'Horažďovice', 'L?pa': 'Lípa', 'Humpolec':'Humpolec', 'Pernolec': 'Pernolec',
                           'Vysok?': 'Vysoká',  'Lukavec': 'Lukavec', 'Vysok? nad Jizerou': 'Vysoké nad Jizerou', '?ab?ice': 'Žabčice', 'Krom????': 'Kroměříž'}, inplace=True)

In [26]:
id_to_site = pd.merge(site_vars, simuid, how = 'left', left_on = ['Site'], right_on = 'Site_Name')

In [28]:
vars_needed = id_to_site[['Site', 'Soil texture', 'LAT', 'LON', 'SIMUID_1']]

In [30]:
# Merge data 
merge = pd.merge(field, vars_needed, how = 'left', on=['Site'])
site_data = pd.merge(merge, tex, how = 'left', on = ['Soil texture'])

In [31]:
site_data = site_data.replace(['winter wheat', 'spring barley ', 'spring barley', 'corn',
       'silage maize'], ['WWHT', 'BARL', 'BARL', 'C100', 'C180'])

In [43]:
keys = [2000,
                                  2001,                            2002,
                                  2003,                            2004,
                                  2005,                            2006,
                                  2007,                            2008,
                                  2009,                            2010,
                                  2011,                            2012,
                                  2013,                            2014,
                                  2015,                            2016,
                                  2017,                            2018]
site_data_long = pd.melt(site_data, id_vars= ['Site','Crop','Experiment','Treatment ','Mineral N kg/ha/y',
                         'farmyard manure t/ha/year','Reported yield data (t DM/ha)', 'Mean','min_yield',
                           'max_yield','Soil texture','LAT','SIMUID_1','Sand','Silt','Clay'],
                         value_vars=keys, value_name='Yield')
site_data_long = site_data_long.drop_duplicates()
site_data_long = site_data_long.dropna( how='any',
                    subset=['Yield'])

In [47]:
site_data

,Site,Crop,Experiment,Treatment,Mineral N kg/ha/y,farmyard manure t/ha/year,Reported yield data (t DM/ha),2000,2001,2002,...,Mean,min_yield,max_yield,Soil texture,LAT,LON,SIMUID_1,Sand,Silt,Clay
0,Kroměříž,WWHT,norf,NaN,100.0,12.5,grain,NaN,5.865000,3.910000,...,7.096438,3.910000,9.477500,loam,49.28,17.36,28776.0,40.0,40.0,20.0
1,Kroměříž,WWHT,norf,NaN,100.0,12.5,grain,NaN,5.865000,3.910000,...,7.096438,3.910000,9.477500,loam,49.28,17.36,28776.0,40.0,40.0,20.0
2,Kroměříž,WWHT,mono,all,150.0,0.0,grain,NaN,4.760000,3.349000,...,5.717313,2.932500,7.480000,loam,49.28,17.36,28776.0,40.0,40.0,20.0
3,Kroměříž,WWHT,mono,all,150.0,0.0,grain,NaN,4.760000,3.349000,...,5.717313,2.932500,7.480000,loam,49.28,17.36,28776.0,40.0,40.0,20.0
4,Kroměříž,BARL,norf,0,0.0,12.5,grain,NaN,3.808000,NaN,...,5.859687,3.808000,7.964500,loam,49.28,17.36,28776.0,40.0,40.0,20.0
5,Kroměříž,BARL,norf,0,0.0,12.5,grain,NaN,3.808000,NaN,...,5.859687,3.808000,7.964500,loam,49.28,17.36,28776.0,40.0,40.0,20.0
6,Kroměříž,BARL,norf,30,30.0,12.5,grain,NaN,4.318000,NaN,...,6.143906,4.318000,7.820000,loam,49.28,17.36,28776.0,40.0,40.0,20.0
7,Kroměříž,BARL,norf,30,30.0,12.5,grain,NaN,4.318000,NaN,...,6.143906,4.318000,7.820000,loam,49.28,17.36,28776.0,40.0,40.0,20.0
8,Kroměříž,BARL,norf,60,60.0,12.5,grain,NaN,4.292500,NaN,...,6.047750,4.292500,7.616000,loam,49.28,17.36,28776.0,40.0,40.0,20.0
9,Kroměříž,BARL,norf,60,60.0,12.5,grain,NaN,4.292500,NaN,...,6.047750,4.292500,7.616000,loam,49.28,17.36,28776.0,40.0,40.0,20.0


### Prep epic data  to be as close to field data as possible

In [31]:
df.columns

Index(['RunID', 'SimUID', 'CROP', 'NFERT', 'BALER', 'YR', 'OCPD', 'TAP',
       'TNO3', 'PRCP', 'PET', 'ET', 'YLDG', 'YLDF', 'BIOM', 'RW', 'FTN', 'FTP',
       'GSET', 'CAW', 'CRF', 'WS', 'NS', 'PS', 'TS', 'PARM', 'SAND', 'SILT',
       'CLAY', 'CLASS', 'WP', 'FWC', 'KS', 'PH'],
      dtype='object')

In [50]:
# pull out the columns needed to reshape dataframe
df_minimal = df[['CROP', 'RunID', 'YR', 'YLDG']]
#reset index
df_minimal.set_index(['RunID', 'CROP'])
# reshape minimal dataframe so that each year is a column with corresponding yield value for that year
df_minimal = df_minimal.pivot_table(index = ['RunID', 'CROP'], columns = 'YR', values = 'YLDG')

In [9]:
# pull out dataframes to take mean of, min of, and max of per RunIDxCROP combo
mean_col = df[['RunID', 'CROP', 'NFERT', 'BALER', 'OCPD', 'TAP', 'SAND', 'SILT',
       'CLAY', 'CLASS', 'PH', 'PRCP', 'YLDG']]
min_col = df[['RunID', 'CROP', 'OCPD', 'TAP', 'PRCP', 'YLDG']]
max_col = df[['RunID', 'CROP', 'OCPD', 'TAP', 'PRCP', 'YLDG']]

In [10]:
# group these dataframes by RunIDxCROP combinations
epic_mean = mean_col.groupby(['RunID', 'CROP', 'CLASS']).mean().reset_index()
epic_min = min_col.groupby(['RunID', 'CROP']).min().reset_index()
epic_max = max_col.groupby(['RunID', 'CROP']).max().reset_index()

In [11]:
#rename columns
epic_mean.columns = ['RunID', 'CROP', 'CLASS', 'mean_NFERT', 'mean_BALER', 'mean_OCPD', 'mean_TAP', 'SAND', 'SILT',
       'CLAY','mean_PH', 'mean_PRCP', 'mean_YLDG']
epic_min.columns = ['RunID', 'CROP', 'min_OCPD', 'min_TAP', 'min_PRCP', 'min_YLDG']
epic_max.columns = ['RunID', 'CROP', 'max_OCPD', 'max_TAP', 'max_PRCP', 'max_YLDG']

In [71]:
#merge all of the above on RunIDxCROP combinations 
epic_1 = pd.merge(epic_mean, epic_min, on=['RunID', 'CROP'])
epic_2 = epic_1.merge(epic_max, on=['RunID', 'CROP'])
epic_2 = epic_2.set_index(['RunID', 'CROP'])
epic = epic_2.merge(df_minimal, left_index = True, right_index = True)
epic = epic.reset_index()

In [13]:
# change column names to make it closer to epic
site_data.columns = [                         'Site',                          'CROP',
                          'Experiment',                    'Treatment ',
                   'NFERT',     'manure',
       'Reported yield data (t DM/ha)',                            '2000',
                                  '2001',                            '2002',
                                  '2003',                            '2004',
                                  '2005',                            '2006',
                                  '2007',                            '2008',
                                  '2009',                            '2010',
                                  '2011',                            '2012',
                                  '2013',                            '2014',
                                  '2015',                            '2016',
                                  '2017',                            '2018',
                                'mean_YLDG',                     'min_YLDG',
                           'max_YLDG',                     'Prec (mm)',
                        'Soil texture',                          'Sand',
                                'Silt',                          'Clay']

In [45]:
site_data.to_csv("C:\\Users\\Research\\Documents\\Tara_Fall_2019\\code\\edited_site_data.csv")

In [46]:
site_data_long.to_csv("C:\\Users\\Research\\Documents\\Tara_Fall_2019\\code\\edited_site_data_long.csv")

In [74]:
epic.to_csv("C:\\Users\\Research\\Documents\\Tara_Fall_2019\\code\\edited_epic_data.csv")